In [ ]:
from geostat import GP, Mesh, NormalizingFeaturizer
import geostat.covfunc as cf
import matplotlib.pyplot as pp
import numpy as np

# Overview

In this notebook we will:
  * Use a Gaussian process to generate synthetic data with known geospatial parameters.
  * Use a second Gaussian process to infer the geospatial parameters from the synthetic data.
  * Use the fitted Gaussian process to interpolate locations on a mesh.

# Synthesizing data

We will synthesize data at mesh locations in a square centered on the origin.

First define mesh locations using a `Mesh` object. The `nx` argument specifies 80 mesh coordinates in the x direction, and keeps the pitch the same in the y direction (which results in 80 mesh coordinates in that direction as well).

In [ ]:
mesh = Mesh.from_bounds([-1, -1, 1, 1], nx=80)

Declare the terms of the spatial trend:

In [ ]:
def trend_terms(x, y): return x, y, x*x, x*y, y*y

Create a featurizer that the Gaussian process class `GP` will use to convert locations into trend features. The method `mesh.locations()` returns an array of shape `[N, 2]`, where `N` is the number of locations. 

In [ ]:
featurizer = NormalizingFeaturizer(trend_terms, mesh.locations())

Instantiate a `GP` and immediately call `generate` to generate synthetic observations.
  * `parameter0` holds the geostatistical parameters for the `GP`.
  * `alpha` parameterizes the normal distribution prior for trend coefficients.
  * `mesh_obs()` has shape `[N]`.

In [ ]:
mesh_obs = GP(
    featurizer = featurizer,
    covariance = cf.SquaredExponential() + cf.Noise(),
    parameters = dict(range=0.33, sill=1., nugget=0.25),
    hyperparameters = dict(alpha=0.2),
    verbose=True).generate(mesh.locations())

When the data is plotted, you can see an overall trend with some localized variations. The method `mesh.slice()` forms the observations into a 2d array suitable for use with `pcolormesh`.

In [ ]:
vmin, vmax = mesh_obs.min(), mesh_obs.max()
meshx, meshy, mesh_obs_2d = mesh.slice(mesh_obs) # Each return value is a 2d array.
c = pp.pcolormesh(meshx, meshy, mesh_obs_2d, vmin=vmin, vmax=vmax)
pp.colorbar(c)
pp.title('Synthetic data')
pp.show()

Of these synthetic datapoints we'll sample just 200, with which we'll try to reconstruct the rest of the data.

In [ ]:
sample_indices = np.random.choice(len(mesh_obs), [200], replace=False)
locs = mesh.locations()[sample_indices, :]
obs = mesh_obs[sample_indices]

c = pp.scatter(locs[:, 0], locs[:, 1], c=obs, vmin=vmin, vmax=vmax)
pp.colorbar(c)
pp.title('Synthetic observations')
pp.show()

# Inferring parameters

Now we create a second `GP`. This time we pass in the data (`locs` and `obs`) and it will fit the geospatial parameters to the data. Here `parameters` holds initial geospatial parameters, which are different that those in the first `GP`, but after fitting they converge to something close.

In [ ]:
gp = GP(featurizer = featurizer,
        covariance = cf.SquaredExponential() + cf.Noise(),
        parameters = dict(range=1.0, sill=0.5, nugget=0.5),
        hyperparameters = dict(alpha=obs.ptp()**2, reg=0, train_iters=300),
        verbose=True).fit(locs, obs)

# Generating predictions

Call `GP` to get predictions at the same mesh locations as before:

In [ ]:
mean, var = gp.predict(locs, obs, mesh.locations())

In [ ]:
meshx, meshy, mean2d = mesh.slice(mean) # Each return value is a 2d array.
c = pp.pcolormesh(meshx, meshy, mean2d, vmin=vmin, vmax=vmax)
pp.colorbar(c)
pp.title('Prediction mean')
pp.show()

For comparison, here's the original synthetic data:

In [ ]:
c = pp.pcolormesh(meshx, meshy, mesh_obs_2d, vmin=vmin, vmax=vmax)
pp.colorbar(c)
pp.title('Synthetic data')
pp.show()

And here's a plot of prediction variance, which accounts for, among other things, the noise that the model is unable to reconstruct.

In [ ]:
meshx, meshy, var2d = mesh.slice(var) # Each return value is a 2d array.
c = pp.pcolormesh(meshx, meshy, var2d, cmap='gist_heat_r')
pp.colorbar(c)
pp.title('Prediction variance')
pp.show()